In [ ]:
import os
import cv2
from pycocotools.coco import COCO
from tqdm import tqdm

def coco_to_yolo(coco_json, img_dir, output_dir):
    os.makedirs(output_dir + "/images/train", exist_ok=True)
    os.makedirs(output_dir + "/labels/train", exist_ok=True)

    coco = COCO(coco_json)
    person_cat_id = coco.getCatIds(catNms=['person'])
    img_ids = coco.getImgIds(catIds=person_cat_id)

    for img_id in tqdm(img_ids, desc="Converting COCO to YOLO"):
        img_info = coco.loadImgs(img_id)[0]
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=person_cat_id)
        anns = coco.loadAnns(ann_ids)

        img_path = os.path.join(img_dir, img_info["file_name"])
        if not os.path.exists(img_path):
            continue

        img = cv2.imread(img_path)
        h, w, _ = img.shape
        out_img_path = os.path.join(output_dir, "images/train", img_info["file_name"])
        cv2.imwrite(out_img_path, img)

        label_path = os.path.join(output_dir, "labels/train", img_info["file_name"].replace(".jpg", ".txt"))
        with open(label_path, "w") as f:
            for ann in anns:
                x, y, bw, bh = ann["bbox"]
                x_center = (x + bw / 2) / w
                y_center = (y + bh / 2) / h
                bw /= w
                bh /= h
                f.write(f"0 {x_center} {y_center} {bw} {bh}\n")

# Run this once to prepare your dataset



In [2]:
coco_to_yolo("annotations/instances_train2017.json", "train2017", "datasets/person")

loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: 'annotations/instances_train2017.json'